In [ ]:
import os
from here_map_widget import Map, LineString, Polygon, Marker, Group
from here_location_services import LS

LS_API_KEY = os.environ.get("LS_API_KEY")

ls = LS(api_key=LS_API_KEY)

result1 = ls.calculate_isoline(
    start=[19.1503, 72.8530],
    range="1800",
    departure="2018-07-16T19:00:00",
    range_type="time",
    mode="fastest;car;traffic:enabled",
)

coordinates1 = result1.isoline[0]["component"][0]["shape"]
line_string1 = LineString()

for latlon in coordinates1:
    x, y = [float(s) for s in latlon.split(",")]
    line_string1.push_point(lat=x, lng=y)

pg1 = Polygon(object=line_string1)

first_center = Marker(lat=19.1503, lng=72.8530)

result2 = ls.calculate_isoline(
    start=[19.1621, 73.0008],
    range="1800",
    departure="2018-07-16T19:00:00",
    range_type="time",
    mode="fastest;car;traffic:enabled",
)

coordinates2 = result2.isoline[0]["component"][0]["shape"]
line_string2 = LineString()

for latlon in coordinates2:
    x, y = [float(s) for s in latlon.split(",")]
    line_string2.push_point(lat=x, lng=y)

pg2 = Polygon(object=line_string2)

second_center = Marker(lat=19.1621, lng=73.0008)

m = Map(
    api_key=LS_API_KEY,
    center=[19.1621, 73.0008],
    zoom=9,
)

In [ ]:
m.add_object(pg1)
m.add_object(first_center)
m

In [ ]:
m.add_object(pg2)
m.add_object(second_center)

In [ ]:
from shapely.geometry import Polygon
import shapely.wkt

l1 = []
for cord in coordinates1:
    l1.append(tuple([float(l) for l in cord.split(",")]))

l2 = []
for cord in coordinates2:
    l2.append(tuple([float(l) for l in cord.split(",")]))


p1 = Polygon(l1)
p2 = Polygon(l2)
p3 = shapely.wkt.loads(str(p1.intersection(p2)))
centroid = p3.centroid
center_marker = Marker(lat=centroid.x, lng=centroid.y)
m

In [ ]:
search = ls.discover(center=[centroid.x, centroid.y], limit=10, query='restaurant', country_codes=["IND"])

In [ ]:
group = Group(volatility=True)

restaurants =[]
for restaurant in search.items:
    restaurants.append(
        Marker(
            lat=restaurant["position"]["lat"],
            lng=restaurant["position"]["lng"],
            data=restaurant["address"]["label"],
        )
    )
group.add_objects(restaurants)
m.add_object(group)